In [161]:
# Routines implementing the variational couple cluster
# method using the LCU method and FPOAA

import numpy as np                          # for rank_1_projector and other custom matrices
import math

from projectq import MainEngine
from projectq.ops import All, H, Ry, X, Y, Z, Measure
from projectq.meta import Dagger, Compute, Uncompute

from projectq.ops._basics import (BasicGate,
                      ClassicalInstructionGate,
                      FastForwardingGate)  
                                            # for projectors and other custom matrices

from projectq.ops import QubitOperator
from fermilib.ops import FermionOperator
from fermilib.transforms import jordan_wigner 
                                            # for Jordan-Wigner transform 
                                            # and fermionic Hamiltonians

# rank-1 projections |i><i| as matrices
# following the pattern in projectq/ops/_gates.py
# __there are probably better ways to do this__
# used in qudit-controlled unitary operations

class Projection(BasicGate):
    """ Projection operator class """
    def __str__(self):
        return "Projection"
     
     
    # constructor for rank-1 projector 
    # dim-qubit operator, projects onto int(reg) = index
    def __init__(self, dim, index):
        size = pow(2, dim)
        if (index >= size):
            print("index out of bounds, throwing exception...")
            return None
        a = np.zeros(size)
        a[index]=1
        self.matrix = np.outer(a, a)
        self.interchangeable_qubit_indices = []

        # providing a decomposition seems to be an important
        # part of defining a gate class
        # provide one possible decomposition :
        register_decomposition(Projection, projection_decomposition1)
        # however this throws a 'name not defined' error

        
class Unitary(BasicGate):
    """ Arbitrary Unitary operator class """
    def __str__(self):
        return "User defined unitary"
     
     
    # constructor for unitary written as \sum_i |i><i|\otimes U_i 
    def __init__(self, matrix):
        self.matrix = matrix
        self.interchangeable_qubit_indices = []


def projector(dim, index):
        size = pow(2, dim)
        if (index >= size):
            print("index out of bounds, throwing exception...")
            return None
        a = np.zeros(size)
        a[index]=1
        matrix = np.outer(a, a)
        return matrix

        
# the LCU 'V' map that prepares the control state
# take the list of 'm' coefts as input
# return the ceil(\log m) qubit register as output

def lcu_control_state_prep(coefts, reg):
    m = len(coefts)
    dim = math.ceil(math.log(m,2)) 
    size = pow(2, dim)
    
    weight = np.sum(coefts)
    probs = np.zeros(size)
    probs[0:m] = coefts
    probs = 1.0/weight * probs 
    
    # compute the rotation angles required for preparing
    # the LCU control state, apply conditional rotations
    # following the method of Grover & Rudolph (2002)
    
    # we need to perform birfurcations over dim rounds
    
    for i in range(1,dim+1):
        block_size = 2**(dim-i+1)
               
        # in each round, need to bifurcate 2^(round - 1) blocks
        
        num_blocks = 2**(i-1)
        target = np.zeros((2**i, 2**i))
        for j in range(0, num_blocks):
            # break loop if we've already crossed the last non-zero coefficient
            if (j*block_size > m):
                break
            
            vec_j = probs[j*block_size : block_size]
            # break loop if singleton
            if (len(vec_j)<=1):
                break
            
            left_cond_prob_j = bifurcate(vec_j)
            ang_j = math.acos(left_cond_prob_j)
            
            proj_j = projector(i-1, j)
            rot = Ry(ang_j).matrix
            temp = np.kron(proj_j, rot)
            target = np.add(target, temp)
        net_rotation = Unitary(target)
        net_rotation | reg[0:i]
    
    return reg

# given a vector, bifurcate it into left and right 
# halves, and return the conditional
# probability of the left half
# will always expect len(vec) a power of 2
def bifurcate(vec):
    m = len(vec)
    tot = np.sum(vec)
    if (tot==0):
        print("some segment has probability 0")
        return 0
    
    left = 0.0
    for i in range (0, m//2):          # // = int division op
        left += vec[i]
        
    return left/tot



# the following function takes a list of unitaries
# and converts them into 
# the controlled unitary \sum_i |i><i| \otimes U_i
# used in the LCU implementation

# include check to make sure that the 
# matrices supplied are unitary

def lcu_controlled_unitary(list_of_U):
    size = len(list_of_U) 
    system_size = len(list_of_U[0].matrix[:,1])              # for square matrices, this hack works
    control_dim = math.ceil(math.log(size, 2))
    mat_dim = 2**control_dim * system_size
    
    target = np.zeros((mat_dim,mat_dim))
    for i in range (0,size):
        proj_i = projector(control_dim, i)
        sysmat = np.asmatrix(list_of_U[i].matrix)
        temp = np.kron(proj_i, sysmat)
        target = np.add(target, temp)
      
    U = Unitary(target)
    return U

# Fixed Point Oblivious Amplitude Amplification (FPOAA)
def fpoaa(reg, proj):
    dim = len(reg)
    size = 2**dim
    id = np.ones(size)
    phi = math.pi/3.0
    quant = math.cos(phi) + math.sin(phi)*1j
    R_target = id - (1-quant) * proj
    return reg


# testing using 2-qubit unitaries
# one possibility: a family of 2-qubit unitaries
# controlled phase
# CP(1,2) = 0.5 * (\id + Z1 + Z2 - Z1 Z2 )

if __name__ == "__main__":
    eng = MainEngine()
    
    dim = 2
        
    A = [X, Z]      # good test because X + Z = H
    c = 1.0/math.sqrt(2)
    coefts = [c, c]
    
    num_1=0
    for i in range(0,100):
        reg = eng.allocate_qureg(dim)
        with Compute(eng):
            lcu_control_state_prep(coefts, reg)
        U = lcu_controlled_unitary(A)
        U | reg
        Uncompute(eng)
        
#         H | reg[1]
#         U = lcu_controlled_unitary(A)
#         U | reg
#         H | reg[1]
        
        Measure | reg[1]
        if (int(reg[1])==0):
            H | reg[0]     # try to uncompute the H applied to qubit 1
            Measure | reg[0]
            num_1 += int(reg[0])
        else:
            All(Measure) | reg

        eng.flush()
#        print("val={}".format(int(anc[0])))
        
    
    print("num of 1 = {}".format(num_1))
    # for i in range(0,dim):
    #    print("{}".format(int(anc[dim-1-i])), end=' ')
    
    
    

num of 1 = 6
